In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sqlalchemy import create_engine
from scipy import stats
import statsmodels.api as sm


import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

weatherinszeged_df = pd.read_sql_query('select * from weatherinszeged',con=engine)

engine.dispose()

weatherinszeged_df.info()
weatherinszeged_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96453 entries, 0 to 96452
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   date                 96453 non-null  datetime64[ns, UTC]
 1   summary              96453 non-null  object             
 2   preciptype           96453 non-null  object             
 3   temperature          96453 non-null  float64            
 4   apparenttemperature  96453 non-null  float64            
 5   humidity             96453 non-null  float64            
 6   windspeed            96453 non-null  float64            
 7   windbearing          96453 non-null  float64            
 8   visibility           96453 non-null  float64            
 9   loudcover            96453 non-null  float64            
 10  pressure             96453 non-null  float64            
 11  dailysummary         96453 non-null  object             
dtypes: datetime64[ns, 

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


#### Build a Regression Model

In [2]:
#Y is the Target
Y = weatherinszeged_df['apparenttemperature'] - weatherinszeged_df['temperature']

#Explanatory Variable
X = weatherinszeged_df[['humidity','windspeed']]

#Add Constant
X = sm.add_constant(X)

In [5]:
#Results
results = sm.OLS(Y,X).fit()

#print results
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Sun, 09 Aug 2020   Prob (F-statistic):               0.00
Time:                        20:04:38   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.000       2.397       2.479
humidity      -3.0292      0.024   -126.479      0.000      -3.076      -2.982
windspeed     -0.1193      0.001   -176.164      0.000      -0.121      -0.118
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.267
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                          -0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### What are the R-squared and adjusted R-squared values? Do you think they are satisfactory? Why?

The R-Squared and Adjusted R-Squared values are 0.288

These values are nor satisfactory as they have only covered approximately 28% of the explained variance with majority unexplained.

#### Next, include the interaction of humidity and windspeed to the model above and estimate the model using OLS. 

In [7]:
weatherinszeged_df['interaction_humidity_windspeed'] = weatherinszeged_df['humidity'] * weatherinszeged_df['windspeed']

Y = weatherinszeged_df['apparenttemperature'] - weatherinszeged_df['temperature']

X = weatherinszeged_df[['humidity', 'windspeed', 'interaction_humidity_windspeed']]

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Sun, 09 Aug 2020   Prob (F-statistic):               0.00
Time:                        20:13:40   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                              0.0839      0.033      2.511      0.012       0.018       0.149
humidity                           0.1775      0.043      4.133      0.000       0.093       0.262
windspeed                          0.0905      0.002     36.797      0.000       0.086       0.095
interaction_humidity_windspeed    -0.2971      0.003    -88.470      0.000      -0.304      -0.291
==============================================================================
Omnibus:                     4849.937   Durbin-Watson:                   0.265
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9295.404
Skew:                          -0.378   Prob(JB):                         0.00
Kurtosis:                       4.320   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### Now, what is the R-squared of this model? 

The R-Squared and Adjusted R-Squared values are now 0.341 or 34%

There was a slight improvement but is still very low. Thus, changes can still be made.

#### Add visibility as an additional explanatory variable to the first model and estimate it. 

In [8]:
weatherinszeged_df['interaction_humidity_windspeed'] = weatherinszeged_df['humidity'] * weatherinszeged_df['windspeed']

Y = weatherinszeged_df['apparenttemperature'] - weatherinszeged_df['temperature']

X = weatherinszeged_df[['humidity', 'windspeed', 'visibility','interaction_humidity_windspeed']]

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.364
Model:                            OLS   Adj. R-squared:                  0.363
Method:                 Least Squares   F-statistic:                 1.377e+04
Date:                Sun, 09 Aug 2020   Prob (F-statistic):               0.00
Time:                        20:16:51   Log-Likelihood:            -1.6504e+05
No. Observations:               96453   AIC:                         3.301e+05
Df Residuals:                   96448   BIC:                         3.301e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                             -1.1006      0.039    -28.459      0.000      -1.176      -1.025
humidity                           0.8909      0.044     20.263      0.000       0.805       0.977
windspeed                          0.1033      0.002     42.579      0.000       0.099       0.108
visibility                         0.0646      0.001     58.051      0.000       0.062       0.067
interaction_humidity_windspeed    -0.3164      0.003    -95.355      0.000      -0.323      -0.310
==============================================================================
Omnibus:                     5328.364   Durbin-Watson:                   0.288
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            11525.074
Skew:                          -0.373   Prob(JB):                         0.00
Kurtosis:                       4.520   Cond. No.                         246.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### Did R-squared increase? What about adjusted R-squared? Compare the differences put on the table by the interaction term and the visibility in terms of the improvement in the adjusted R-squared. Which one is more useful?

Yes, the R-squared and adjusted R-squared increased.

By adding visiblity, the R-squared and adjusted R-squared to .364 or 36% of variance explained. This is higher and a better number compared to 0.341 when visibility is not included and an additional explanatory variable.

#### Choose the best one from the three models above with respect to their AIC and BIC scores. Validate your choice by discussing your justification with your mentor.

The best model from the 3 models created above contains the 4 explanatory variables, 'humidity', 'windspeed', 'visibility','interaction_humidity_windspeed'.

This is because that this model also has the lowest AIC and BIC score of 3.301 on both.